<a href="https://colab.research.google.com/github/BentonMiller/python1/blob/master/Simplified_Quant_NoteBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Specify a path to your files here:**

 *A Sample Sheet must have the columns 'well_position', 'name', and 'concentration'.*

 *A name that begins with 'Std' [note the capitalization] will be used as a calibration curve data point.*

 *The name 'background' will be used for background subtraction. If no background is present in the samplesheet, background will be 0.*



In [67]:
data_file_path = '/content/drive/Shareddrives/Leash - Shared/Operations/Data/Quant Data Files and Sample Sheets/2024-02-21 09-37-38_plate_1.xml'
sample_sheet_share_url = 'https://docs.google.com/spreadsheets/d/1d94lDtCDDthZx4_5PF7JqLYYAe884KTlEOIfBzcxBFc/edit#gid=0'


In [71]:
# @title Process the Files.

from google.colab import drive
from xml.dom import minidom
import pandas as pd
pd.options.mode.chained_assignment = None # ignore warnings from pandas
from scipy import stats
import plotly.express as px

# Mount Google Drive and get content of XML FIle:
drive.mount('/content/drive')
wells = minidom.parse(data_file_path).getElementsByTagName("Well")
data = []
for well in wells:
    well_position = well.getAttribute("Pos")
    measurement = list(well.getElementsByTagName("Single"))
    for single in measurement:
        value = float(single.firstChild.nodeValue)
        row = {'well_position':well_position, 'rfu_value':value}
        data.append(row)
print("{0} wells of data extracted from XML file: {1}".format(len(data),data_file_path))

# Connect to Google Sheets and get content of Sample Sheet:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_url(sample_sheet_share_url)
worksheet = workbook.sheet1
sample_sheet = filter(None,worksheet.get_all_records())

# Join Sample Sheet and XML Data Together:
sample_sheet_rows = []
for row in sample_sheet:
  sample_sheet_rows.append(row)
sample_sheet_df = pd.DataFrame(sample_sheet_rows)
xml_df = pd.DataFrame(data)
data_df = xml_df.set_index('well_position').join(sample_sheet_df.set_index('well_position'))

# Find Standards:
standards = data_df[data_df['name'].str.contains('Std', na=False)]
background = None
background = data_df[data_df['name'].str.contains('background', na=False)]
samples = data_df[~data_df['name'].str.contains('Std', na=False)]
x_range = standards['rfu_value']
y_range = standards['concentration']
if background is None:
  background = 0
else:
  background = background['rfu_value'].tolist()[0]

# Calculate best fit line:
x_subrange = x_range.to_list()
y_subrange = y_range.to_list()
x_subrange_bg_subtract = [x-background for x in x_subrange]
slope,intercept,r,tt,stderr=stats.linregress(x_subrange_bg_subtract,y_subrange)
print("Background = {3} The linear equation for the fit is y = {0:1.4E} x + {1:1.4E}, with an R-squared value of {2:1.5f}.".format(slope, intercept, r**2, background))

# Calculate concentrations for samples here...
samples['concentration'] = samples['rfu_value'].map(lambda rfu_value: slope*rfu_value+intercept )
samples['type'] = 'Sample'
standards['type'] = 'Standard'

# Add standards into Sample dataset
samples = pd.concat([samples,standards])

# Display calibration curve plot:
fig = px.scatter(samples,x="rfu_value",y="concentration", color='type', hover_name="name", hover_data={"well_position": (samples.index)})
fig.show()

# Calculate row_num and col for platemap
samples['row'] = samples.index
import re
samples['col'] = samples['row']
samples['col'] = samples['row'].map(lambda row: int(re.search('[0-9]+',row).group(0)))
samples['row'] = samples['row'].map(lambda row: re.search('[A-Z]+',row).group(0))

import string
samples['row_num'] = samples['row'].map(lambda row: string.ascii_uppercase.index(row)+1)
samples = samples.fillna('')
samples = samples.reset_index()
samples = samples.sort_values(["row_num","col"], ascending=True)
df_col_values = samples.values.tolist()
try:
  data_sheet = workbook.worksheet('data_sheet')
  print("Appending to current data_sheet")
except:
  data_sheet = workbook.add_worksheet(title="data_sheet", rows=samples.shape[0], cols=samples.shape[1])
  print("Creating new data_sheet")

data_sheet.append_row(["Data from:"])
data_sheet.append_row([data_file_path])
data_sheet.append_rows([samples.columns.values.tolist()] + df_col_values)
samples.pivot_table(index="row",columns="col",values="concentration").style.background_gradient(axis=1, cmap='GnBu')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
384 wells of data extracted from XML file: /content/drive/Shareddrives/Leash - Shared/Operations/Data/Quant Data Files and Sample Sheets/2024-02-21 09-37-38_plate_1.xml
Background = 18.0 The linear equation for the fit is y = 2.2745E-03 x + 4.4931E-01, with an R-squared value of 0.99852.


Appending to current data_sheet


col,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
row,,,,,,,,,,,,,,,,,,,,,,,,
A,100.000000,0.526642,50.000000,0.508446,25.000000,0.492525,12.500000,0.483427,6.250000,0.483427,3.125000,0.481152,1.562500,0.481152,0.000000,0.481152,0.808677,0.481152,0.806402,0.476603,0.824598,0.478878,0.842794,0.478878
B,0.526642,0.815500,0.524367,0.788207,0.499348,0.783658,0.490250,0.751815,0.485701,0.801854,0.485701,0.774560,0.481152,0.788207,0.481152,0.788207,0.483427,0.795030,0.481152,0.767736,0.481152,0.833696,0.478878,0.790481
C,9.212867,0.490250,6.092286,0.492525,16.548054,0.487976,4.909558,0.485701,9.349336,0.483427,6.927019,0.487976,14.539691,0.487976,0.845069,0.485701,9.444864,0.483427,5.446334,0.483427,7.347797,0.483427,6.322008,0.483427
D,0.490250,0.838245,0.487976,0.779109,0.492525,0.799579,0.487976,0.758638,0.483427,0.815500,0.483427,0.824598,0.490250,0.806402,0.485701,6.708669,0.485701,0.692679,0.485701,0.694953,0.481152,0.715423,0.485701,0.719972
E,11.287190,0.487976,8.751149,0.490250,8.610131,0.485701,0.733619,0.478878,0.478878,0.478878,0.481152,0.476603,0.476603,0.478878,0.478878,0.483427,0.478878,0.481152,0.478878,0.481152,0.481152,0.481152,0.485701,0.487976
F,0.490250,0.747266,0.485701,0.760913,0.487976,0.738168,0.476603,0.742717,0.476603,0.758638,0.478878,0.770011,0.478878,0.733619,0.478878,0.749541,0.478878,0.776834,0.481152,0.788207,0.481152,0.756364,0.485701,0.795030
G,0.481152,0.478878,0.478878,0.476603,0.478878,0.478878,0.478878,0.474329,0.478878,0.478878,0.478878,0.476603,0.478878,0.481152,0.478878,0.476603,0.476603,0.476603,0.481152,0.499348,0.478878,0.478878,0.542563,0.481152
H,0.478878,0.476603,0.476603,0.478878,0.481152,0.478878,0.478878,0.478878,0.476603,0.478878,0.478878,0.476603,0.478878,0.476603,0.478878,0.474329,0.481152,0.481152,0.485701,0.503897,0.476603,0.478878,0.510721,0.487976
I,0.478878,0.478878,0.476603,0.478878,0.481152,0.476603,0.478878,0.474329,0.478878,0.476603,0.478878,0.478878,0.476603,0.476603,0.476603,0.476603,0.478878,0.478878,0.481152,0.499348,0.481152,0.478878,0.476603,0.483427


In [72]:
samples.pivot_table(index="row",columns="col",values="rfu_value").style.background_gradient(axis=1, cmap='GnBu')

col,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
row,,,,,,,,,,,,,,,,,,,,,,,,
A,43286.000000,34.000000,23048.000000,26.000000,10395.000000,19.000000,4899.000000,15.000000,2205.000000,15.000000,1264.000000,14.000000,555.000000,14.000000,157.000000,14.000000,158.000000,14.000000,157.000000,12.000000,165.000000,13.000000,173.000000,13.000000
B,34.000000,161.000000,33.000000,149.000000,22.000000,147.000000,18.000000,133.000000,16.000000,155.000000,16.000000,143.000000,14.000000,149.000000,14.000000,149.000000,15.000000,152.000000,14.000000,140.000000,14.000000,169.000000,13.000000,150.000000
C,3853.000000,18.000000,2481.000000,19.000000,7078.000000,17.000000,1961.000000,16.000000,3913.000000,15.000000,2848.000000,17.000000,6195.000000,17.000000,174.000000,16.000000,3955.000000,15.000000,2197.000000,15.000000,3033.000000,15.000000,2582.000000,15.000000
D,18.000000,171.000000,17.000000,145.000000,19.000000,154.000000,17.000000,136.000000,15.000000,161.000000,15.000000,165.000000,18.000000,157.000000,16.000000,2752.000000,16.000000,107.000000,16.000000,108.000000,14.000000,117.000000,16.000000,119.000000
E,4765.000000,17.000000,3650.000000,18.000000,3588.000000,16.000000,125.000000,13.000000,13.000000,13.000000,14.000000,12.000000,12.000000,13.000000,13.000000,15.000000,13.000000,14.000000,13.000000,14.000000,14.000000,14.000000,16.000000,17.000000
F,18.000000,131.000000,16.000000,137.000000,17.000000,127.000000,12.000000,129.000000,12.000000,136.000000,13.000000,141.000000,13.000000,125.000000,13.000000,132.000000,13.000000,144.000000,14.000000,149.000000,14.000000,135.000000,16.000000,152.000000
G,14.000000,13.000000,13.000000,12.000000,13.000000,13.000000,13.000000,11.000000,13.000000,13.000000,13.000000,12.000000,13.000000,14.000000,13.000000,12.000000,12.000000,12.000000,14.000000,22.000000,13.000000,13.000000,41.000000,14.000000
H,13.000000,12.000000,12.000000,13.000000,14.000000,13.000000,13.000000,13.000000,12.000000,13.000000,13.000000,12.000000,13.000000,12.000000,13.000000,11.000000,14.000000,14.000000,16.000000,24.000000,12.000000,13.000000,27.000000,17.000000
I,13.000000,13.000000,12.000000,13.000000,14.000000,12.000000,13.000000,11.000000,13.000000,12.000000,13.000000,13.000000,12.000000,12.000000,12.000000,12.000000,13.000000,13.000000,14.000000,22.000000,14.000000,13.000000,12.000000,15.000000
